In [5]:
import os

if os.path.exists('input.mp4'):
    size = os.path.getsize('input.mp4')
    print(f"input.mp4 găsit! Dimensiune: {size / 1e6:.1f} MB")
else:
    print("input.mp4 NU e în folder! Pune-l lângă notebook.")

input.mp4 găsit! Dimensiune: 112.1 MB


In [7]:
def hexdump(file_path, lines=20):
    with open(file_path, 'rb') as f:
        for i in range(lines):
            data = f.read(16)
            if not data: break
            hex_part = ' '.join(f'{b:02x}' for b in data)
            ascii_part = ''.join(chr(b) if 32 <= b < 127 else '.' for b in data)
            print(f"{i*16:08x}  {hex_part:<48}  {ascii_part}")

hexdump('input.mp4')

00000000  00 00 00 20 66 74 79 70 69 73 6f 6d 00 00 02 00   ... ftypisom....
00000010  69 73 6f 6d 69 73 6f 32 61 76 63 31 6d 70 34 31   isomiso2avc1mp41
00000020  00 00 00 08 66 72 65 65 06 8f 7c 6d 6d 64 61 74   ....free..|mmdat
00000030  00 00 02 ae 06 05 ff ff aa dc 45 e9 bd e6 d9 48   ..........E....H
00000040  b7 96 2c d8 20 d9 23 ee ef 78 32 36 34 20 2d 20   ..,. .#..x264 - 
00000050  63 6f 72 65 20 31 36 34 20 72 33 31 30 38 20 33   core 164 r3108 3
00000060  31 65 31 39 66 39 20 2d 20 48 2e 32 36 34 2f 4d   1e19f9 - H.264/M
00000070  50 45 47 2d 34 20 41 56 43 20 63 6f 64 65 63 20   PEG-4 AVC codec 
00000080  2d 20 43 6f 70 79 6c 65 66 74 20 32 30 30 33 2d   - Copyleft 2003-
00000090  32 30 32 33 20 2d 20 68 74 74 70 3a 2f 2f 77 77   2023 - http://ww
000000a0  77 2e 76 69 64 65 6f 6c 61 6e 2e 6f 72 67 2f 78   w.videolan.org/x
000000b0  32 36 34 2e 68 74 6d 6c 20 2d 20 6f 70 74 69 6f   264.html - optio
000000c0  6e 73 3a 20 63 61 62 61 63 3d 31 20 72 65 66 3d   ns: cabac=1 ref=

In [11]:
import subprocess
from IPython.display import Audio

print("Extragem audio din video...")

result = subprocess.run([
    'ffmpeg', '-y', '-i', 'input.mp4', 'audio.wav'
], capture_output=True, text=True)

if result.returncode == 0:
    print("AUDIO EXTRAS: audio.wav")
    print("Ascultă-l acum!")
    display(Audio('audio.wav'))
else:
    print("EROARE:")
    print(result.stderr)

Extragem audio din video...
AUDIO EXTRAS: audio.wav
Ascultă-l acum!


In [1]:
!pip install openai-whisper

In [1]:
!pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [3]:
from pydub import AudioSegment
import os

# Încarcă audio.wav
audio = AudioSegment.from_wav("audio.wav")

# Definește timpurile (în milisecunde)
times = [
    (45 * 1000, 55 * 1000),      # 0:45 → 0:55
    (25*60*1000 + 10*1000, 25*60*1000 + 20*1000),  # 25:10 → 25:20
    (49*60*1000 + 30*1000, 49*60*1000 + 40*1000)   # 49:30 → 49:40
]

# Extrage și salvează 3 bucăți
for i, (start, end) in enumerate(times):
    clip = audio[start:end]
    clip.export(f"clip_{i+1}.wav", format="wav")
    print(f"clip_{i+1}.wav salvat: {start//1000} sec → {end//1000} sec")

print("\n3 CLIPURI CREATE! Fiecare = 10 secunde")

clip_1.wav salvat: 45 sec → 55 sec
clip_2.wav salvat: 1510 sec → 1520 sec
clip_3.wav salvat: 2970 sec → 2980 sec

3 CLIPURI CREATE! Fiecare = 10 secunde


In [5]:
import whisper
model = whisper.load_model("tiny")

print("Transcriu doar 3 clipuri de 10 secunde...")

transcripts = []
for i in range(1, 4):
    result = model.transcribe(f"clip_{i}.wav", language="en")
    text = result["text"].strip()
    print(f"Clip {i}: {text}")
    transcripts.append(text)

print("\nTOATE TRANSCRIPT-URILE:")
print(transcripts)

100%|█████████████████████████████████████| 72.1M/72.1M [01:08<00:00, 1.10MiB/s]


Transcriu doar 3 clipuri de 10 secunde...
Clip 1: We're gonna let you down, we're gonna run around here You just let you, we're gonna make you cry We're gonna shake it back
Clip 2: We're no strangers to love You know the rules and so do I After the commitments fall
Clip 3: You're a woman, you're a teacher I'm a men you are You're a good woman, say goodbye I'm a good woman, say goodbye

TOATE TRANSCRIPT-URILE:
["We're gonna let you down, we're gonna run around here You just let you, we're gonna make you cry We're gonna shake it back", "We're no strangers to love You know the rules and so do I After the commitments fall", "You're a woman, you're a teacher I'm a men you are You're a good woman, say goodbye I'm a good woman, say goodbye"]


In [7]:
import re

full = " ".join(transcripts)

# Caută FLAG-ul
flag = re.search(r'LMML\{[^}]*\}', full, re.IGNORECASE)
if flag:
    print("\nFLAG GĂSIT!")
    print(f"   {flag.group(0)}")
    print(f"   TRIMITE ACUM: {flag.group(0)}")
else:
    # Manual: lipește părțile
    print("\nLipește manual:")
    print(" + ".join(transcripts))
    print(f"POSIBIL FLAG: LMML" + "".join([t.replace(" ", "") for t in transcripts[1:]]))


Lipește manual:
We're gonna let you down, we're gonna run around here You just let you, we're gonna make you cry We're gonna shake it back + We're no strangers to love You know the rules and so do I After the commitments fall + You're a woman, you're a teacher I'm a men you are You're a good woman, say goodbye I'm a good woman, say goodbye
POSIBIL FLAG: LMMLWe'renostrangerstoloveYouknowtherulesandsodoIAfterthecommitmentsfallYou'reawoman,you'reateacherI'mamenyouareYou'reagoodwoman,saygoodbyeI'magoodwoman,saygoodbye
